In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException
import time
import re
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import random

In [2]:
# Chrome 설정
def ChromeSetup():
    options = webdriver.ChromeOptions()
    options.add_argument("headless")  # 백그라운드 실행
    options.add_argument("--ignore-ssl-errors=yes")
    options.add_argument("--ignore-certificate-errors")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-popup-blocking")
    # Random User-Agent 사용하여 차단 회피
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:85.0) Gecko/20100101 Firefox/85.0",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36",
    ]
    options.add_argument(f"user-agent={random.choice(user_agents)}")
    return options

In [3]:
# 데이터 로드
number = pd.read_csv("../data/lodging_place_number(kakao).csv")

# 전체 데이터 저장할 리스트 및 오류 데이터 리스트 설정
all_data = []  # 전체 리뷰 데이터를 저장할 리스트
error_data = []  # 오류가 발생한 숙소 정보 저장할 리스트

In [4]:
# 드라이버 4개 생성
driver_pool = [webdriver.Chrome(options=ChromeSetup()) for _ in range(1)]

In [5]:
def collect_reviews(row, driver):
    lodging_id = row["lodging_id"]
    url = row["href"]
    local_data = []

    try:
        driver.get(url)

        # 랜덤 시간 동안 대기하여 패턴 감지 방지
        time.sleep(random.uniform(1, 7))

        # 페이지가 완전히 로드될 때까지 대기 (필요한 요소가 로드될 때까지 기다림)
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, "location_evaluation")))

        # 리뷰 개수 확인 및 수집 여부 결정
        review_count = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "location_evaluation"))
        )

        # 리뷰 개수 및 상태 확인
        review_text = None
        try:
            review_no = review_count.find_element(By.CLASS_NAME, 'ico_noti').text
            if review_no == "후기미제공":
                return
        except NoSuchElementException:
            review_text = review_count.find_element(By.CLASS_NAME, "color_g").text

        if review_text == "(0)":
            return

        # '후기 더보기' 버튼 클릭하여 모든 리뷰 로드
        while True:
            try:
                button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "link_more")))
                if "후기 더보기" in button.text:
                    button.click()
                    time.sleep(random.uniform(1, 3))  # 비동기 로딩 시간 확보
                else:
                    break
            except (StaleElementReferenceException, TimeoutException, NoSuchElementException):
                break

        # 리뷰 리스트에서 리뷰 정보 수집
        review_list = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "list_evaluation"))
        )
        li_elements = review_list.find_elements(By.XPATH, "./li")

        # 각 리뷰의 정보 추출
        for li in li_elements:
            local_data.append(
                {
                    "lodging_id": lodging_id,
                    "rating": extract_rating(li, url),
                    "review_text": extract_review_text(li, url),
                    "photos": extract_photos(li, url) or None,
                    "date": extract_review_date(li, url),
                }
            )

        # 리뷰 수집 성공 시 공용 리스트에 추가
        all_data.extend(local_data)

    except TimeoutException:
        error_data.append({"lodging_id": lodging_id, "href": url})


In [6]:
# 별점 추출 함수
def extract_rating(li, url):
    try:
        star = WebDriverWait(li, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "inner_star")))
        style = star.get_attribute("style")
        width_match = re.search(r"width:\s*(\d+)%", style)
        return int(width_match.group(1)) // 20 if width_match else 0
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        print(f"별점 정보 추출 실패, {url}")
        return 0

In [7]:
# 사진 정보 추출 함수
def extract_photos(li, url):
    photos = []
    try:
        # 요소가 존재하는지 먼저 확인
        photo_elements = li.find_elements(By.CLASS_NAME, "list_photo")
        if not photo_elements:
            return photos

        # 요소가 존재하면 사진을 추출
        photo_container = photo_elements[0]
        photo_li_elements = photo_container.find_elements(By.TAG_NAME, "li")
        for photo_li in photo_li_elements[:5]:  # 최대 5개의 사진만 저장
            img_tag = photo_li.find_element(By.TAG_NAME, "img")
            photos.append(img_tag.get_attribute("src"))

    except (NoSuchElementException, StaleElementReferenceException):
        pass
    except TimeoutException:
        print(f"사진 정보 추출 실패, {url}")
    return photos

In [8]:
# 리뷰 텍스트 추출 함수
def extract_review_text(li, url):
    try:
        comment = WebDriverWait(li, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "txt_comment")))
        if comment:
            span = comment.find_element(By.TAG_NAME, "span")
            review_text = span.text if span.text else ""
            return review_text
        else:
            return ""
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        print(f"리뷰 텍스트 추출 실패, {url}")
        return ""

In [9]:
# 리뷰 날짜 추출 함수
def extract_review_date(li, url):
    try:
        date = WebDriverWait(li, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "time_write"))).text
        return date
    except (NoSuchElementException, StaleElementReferenceException, TimeoutException):
        print(f"리뷰 날짜 추출 실패, {url}")
        return ""

In [10]:
# 멀티쓰레드로 리뷰 수집 실행
with ThreadPoolExecutor(max_workers=1) as executor:
    future_to_row = {
        executor.submit(collect_reviews, row[1], driver_pool[i % 1]): row for i, row in enumerate(number.iterrows())
    }
    with tqdm(total=len(number), desc="리뷰 수집 중", position=0) as pbar:
        for future in as_completed(future_to_row):
            pbar.update(1)

리뷰 수집 중: 100%|██████████| 2922/2922 [11:24:50<00:00, 14.06s/it]    


In [ ]:
# 전체 데이터프레임으로 변환
if all_data:
    df = pd.DataFrame(all_data)
    df.to_csv("../data/lodging_reviews.csv", index=False, encoding="utf-8-sig")

In [13]:
# 오류 데이터 재시도 로직 추가
retry_limit = 3  # 재시도 한계를 3번으로 설정
retry_count = 0
while error_data and retry_count < retry_limit:
    print(f"Retrying errors... Attempt {retry_count + 1}")
    retry_count += 1

    # 새로운 드라이버 세션 시작
    driver = webdriver.Chrome(options=ChromeSetup())

    # 오류 데이터 수집 재시도
    temp_error_data = []  # 이번 시도에서 실패한 데이터 저장
    with tqdm(total=len(error_data), desc="Retrying Errors", position=0, leave=True) as pbar:
        for error_row in error_data:
            try:
                collect_reviews(pd.Series(error_row), driver)
            except Exception as e:
                temp_error_data.append(error_row)  # 이번 시도에서 실패한 데이터 유지
            pbar.update(1)

    # 남은 오류 데이터를 업데이트 (성공한 데이터를 제외한 나머지를 유지)
    error_data = temp_error_data

    # 드라이버 종료
    driver.quit()

Retrying errors... Attempt 1


Retrying Errors:   0%|          | 0/219 [00:11<?, ?it/s]


KeyboardInterrupt: 

In [14]:
error_data

[{'lodging_id': 68, 'href': 'https://place.map.kakao.com/12697155'},
 {'lodging_id': 447, 'href': 'https://place.map.kakao.com/1993599504'},
 {'lodging_id': 452, 'href': 'https://place.map.kakao.com/233994951'},
 {'lodging_id': 455, 'href': 'https://place.map.kakao.com/16260352'},
 {'lodging_id': 486, 'href': 'https://place.map.kakao.com/17735912'},
 {'lodging_id': 506, 'href': 'https://place.map.kakao.com/7817921'},
 {'lodging_id': 526, 'href': 'https://place.map.kakao.com/12487084'},
 {'lodging_id': 786, 'href': 'https://place.map.kakao.com/1554018549'},
 {'lodging_id': 795, 'href': 'https://place.map.kakao.com/8954246'},
 {'lodging_id': 1163, 'href': 'https://place.map.kakao.com/14742749'},
 {'lodging_id': 1176, 'href': 'https://place.map.kakao.com/27317836'},
 {'lodging_id': 1292, 'href': 'https://place.map.kakao.com/1290040691'},
 {'lodging_id': 1336, 'href': 'https://place.map.kakao.com/26772861'},
 {'lodging_id': 1375, 'href': 'https://place.map.kakao.com/16279260'},
 {'lodging_

In [ ]:
# 드라이버 종료
for driver in driver_pool:
    driver.quit()

In [11]:
# 전체 데이터프레임으로 변환
if all_data:
    df = pd.DataFrame(all_data)
    df.to_csv("../data/lodging_reviews(final).csv", index=False, encoding="utf-8-sig")